# Day 12: GUI Development

## Build Web Interfaces for Your Agents

Today: Create beautiful UIs with WebUI!

### Topics:
1. WebUI basics
2. chatbot_config options
3. File uploads
4. Gradio customization
5. Deployment

In [ ]:
import os
os.environ['FIREWORKS_API_KEY'] = 'fw_3ZTLPrnEtuscTUPYy3sYx3ag'
llm_cfg = {'model': 'accounts/fireworks/models/qwen3-235b-a22b-thinking-2507', 'model_server': 'https://api.fireworks.ai/inference/v1', 'api_key': os.environ['FIREWORKS_API_KEY']}
print('✅ Configured')

## Basic WebUI

```python
from qwen_agent.agents import Assistant
from qwen_agent.gui import WebUI

bot = Assistant(llm=llm_cfg)

chatbot_config = {
    'prompt.suggestions': [
        'Hello',
        'Tell me a joke'
    ]
}

WebUI(bot, chatbot_config=chatbot_config).run()
```

Launches a web interface!

## File Uploads

Enable file uploads for RAG:
```python
bot = Assistant(llm=llm_cfg, files=[])
WebUI(bot).run()
```

Users can upload documents in the UI.

## Deployment

1. **Local**: `WebUI(bot).run()`
2. **Share link**: `WebUI(bot).run(share=True)`
3. **Docker**: Use Gradio Docker images
4. **Production**: Add authentication, HTTPS

## 🎉 Course Complete!

You've mastered:
- Days 1-3: Foundations
- Days 4-7: Tools and agents
- Days 8-10: Advanced features
- Days 11-12: Production patterns

**You're ready to build production AI agents!** 🚀